# Email Process Automation with AI

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
#from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import resend
import requests

In [2]:
load_dotenv(override=True)
RESEND_API_KEY = os.getenv("RESEND_API_KEY")

In [3]:
resend.api_key = RESEND_API_KEY

r = resend.Emails.send({
  "from": "test@resend.dev",
  "to": "krzysztof.satola@gmail.com",
  "subject": "Hello World",
  "html": "<p>Congrats on sending your <strong>first email</strong>!</p>"
})

## Step 1: Agent workflow

In [4]:
instructions1 = "You are a sales agent working for BikeRent, \
a company that provides bikes for share in Cracow. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for BikeRent, \
a company that provides bikes for share in Cracow. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for BikeRent, \
a company that provides bikes for share in Cracow. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
# If using different OpenAI API compatible models, 
# the following code should be used (the OpenAI SDK works the same)

from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel

google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [6]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for BikeRent, a company that provides bikes for share in Cracow. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [7]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Enhance Your Travel Experience in Cracow with BikeRent

Dear [Recipient's Name],

I hope this message finds you well.

I am reaching out to introduce you to BikeRent, a premier bike-sharing service in Cracow designed to enhance your experience in our vibrant city. As more visitors seek sustainable and flexible transportation options, our service offers a convenient solution that promotes both wellness and exploration.

With our easy-to-use app, you can quickly find and rent a bike, allowing you to navigate the city at your own pace while taking in the beautiful sights of Cracow. Our fleet includes a variety of bikes to suit all needs, from leisurely rides to more active adventures.

**Key Benefits of Partnering with BikeRent:**

- **Flexibility:** Rent for a few hours or several days, tailored to your schedule.
- **Convenience:** Multiple rental stations located throughout the city.
- **Sustainability:** Encourage eco-friendly travel and reduce carbon footprints.

I would love

In [8]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Elevate Your Business with BikeRent Solutions

Dear [Recipient's Name],

I hope this message finds you well.

My name is [Your Name], and I am reaching out from BikeRent, a premier bike-sharing service operating in Cracow. As cities increasingly embrace sustainable transportation options, we believe that our services can significantly enhance your business offerings and community engagement.

At BikeRent, we provide a seamless bike-sharing solution that can be easily integrated into various sectors, from tourism and hospitality to corporate campuses. Our user-friendly platform offers customers an eco-friendly mode of transportation, promoting health, sustainability, and local exploration.

Here are a few benefits of partnering with us:

1. **Increased Foot Traffic:** Attract more customers to your location by promoting bike rentals as a convenient travel option.
2. **Customized Solutions:** Tailored packages to fit your specific needs and enhance your branding.
3. **Sustainabl

In [9]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [10]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Ready to Pedal into Adventure? 🚴‍♂️

Hey [Recipient's Name]! 

Are you tired of being stuck in traffic, clinging to your coffee like a lifeboat in a storm? 🚦☕

What if I told you there's a magical way to glide through Cracow’s historic streets, with the wind in your hair and a smile on your face? Introducing BikeRent – your new best friend for two-wheeled adventures!

Imagine this: you, a comfy bike, and the breathtaking sights of the Wawel Castle. No better way to impress your friends than to say you “biked past royalty!”

Here are some *pedal-tastic* reasons to join the BikeRent family:  
- Super easy rental process (you don’t even need to wear spandex!). 
- Flexible rental periods – whether you're out for a leisurely Sunday ride or an adrenaline-packed commute, we’ve got you covered. 
- A fleet of bikes that won’t judge you for that extra slice of pizza. 🍕

Curious to learn more? I promise it’ll be worth it! Just reply to this email, and I’ll get back to y

https://platform.openai.com/traces

## Step 2: Tools and Agent interactions

In [11]:
@function_tool
def send_email(subject: str="Sales email", body: str="") -> Dict[str, str]:
    """ Send out an email with the given body to all sales prospects """
    resend.api_key = RESEND_API_KEY

    from_email = "test@resend.dev"
    to_email = ["krzysztof.satola@gmail.com"]
    content = f"<p>{body}</p>"

    resp = resend.Emails.send({
        "from": from_email,
        "to": to_email,
        "subject": subject,
        "html": content
        })
    return resp

In [12]:
@function_tool
def send_email2(subject: str="Sales email", body: str="") -> Dict[str, str]:
    """ Send out an email with the given body to all sales prospects via Resend """
    
    from_email = "test@resend.dev"
    to_email = "krzysztof.satola@gmail.com"
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"Test <{from_email}>",
        "to": [to_email],
        "subject": subject,
        "html": f"<p>{body}</p>"
    }
    
    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
    
    if response.status_code == 202:
        return {"status": "success"}
    else:
        return {"status": "failure", "message": response.text}

In [13]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'subject': {'default': 'Sales email', 'title': 'Subject', 'type': 'string'}, 'body': {'default': '', 'title': 'Body', 'type': 'string'}}, 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False, 'required': ['subject', 'body']}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f4d6fdf0220>, strict_json_schema=True, is_enabled=True)

In [14]:
# Convert the agent to a tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f4d6ecae0c0>, strict_json_schema=True, is_enabled=True)

In [15]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

#tools

In [16]:
# Sales manager agent that uses the sales agents and the email sender tool
instructions ="You are a sales manager working for BikeRent. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)

#https://platform.openai.com/logs?api=traces

### Handoffs
Represent a way an agent can delegate to an agent, passing control to it. Handoffs and Agents-as-tools are similar:
- In both cases, an Agent can collaborate with another Agent
- With **tools**, control passes back
- With **handoffs**, control passes across

In [17]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [18]:
# Email manager agent that creates the subject, HTML body for the email and sends it out
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [19]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f4d6ecae160>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f4d6ecae200>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

## Complete workflow

In [20]:
sales_manager_instructions = "You are a sales manager working for BikeRent. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Krzysztof"

with trace("Automated Cold Email Campaign"):
    result = await Runner.run(sales_manager, message)

#https://platform.openai.com/traces

## Complete workflow with a Guardrail
There can be input and output guardrails

In [21]:
from pydantic import BaseModel
from agents import input_guardrail, GuardrailFunctionOutput

class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [22]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)

In [23]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated Cold Email Campaign"):
    result = await Runner.run(careful_sales_manager, message)

#https://platform.openai.com/traces

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [24]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated Cold Email Campaign"):
    result = await Runner.run(careful_sales_manager, message)